<a href="https://colab.research.google.com/github/ryan840919/STAS-Detection/blob/main/Prediction_and_Ground_true.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
filename = os.listdir('/content/gdrive/MyDrive/AI競賽_運算/data/STAS_YOLOv5_PS/images/val')
numlist = []
for fil in filename:
  name = fil.split('.')
  numlist.append(int(name[0]))
print(numlist)

[288, 275, 439, 229, 847, 571, 737, 998, 164, 242, 983, 482, 2, 872, 216, 889, 616, 719, 72, 177, 253, 319, 430, 487, 386, 984, 987, 360, 305, 768, 140, 718, 659, 359, 575, 852, 600, 494, 918, 318, 637, 666, 381, 563, 946, 266, 832, 256, 163, 511, 744, 78, 349, 869, 76, 378, 301, 421, 1019, 937, 265, 350, 686, 633, 260, 539, 1048, 490, 842, 285, 897, 845, 375, 800, 862, 199, 510, 86, 724, 429, 653, 513, 58, 546, 180, 870, 740, 355, 1044, 83, 948, 757, 479, 551, 694, 682, 856, 726, 710, 12, 534, 699, 594, 240, 596]


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

def P_G(test_name):
  os.makedirs(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/{test_name}')
  for num in numlist:
    with open(f'/content/gdrive/MyDrive/AI競賽_運算/yolov5/runs/val/{test_name}/labels/{num:08}.txt','r') as R:
      corlis = []
      for line in R.readlines():
        word = line.split(' ')
        xmin = round(float(word[1])*1716-float(word[3])*1716/2)
        ymin = round(float(word[2])*942-float(word[4])*942/2)
        xmax = round(float(word[1])*1716+float(word[3])*1716/2)
        ymax = round(float(word[2])*942+float(word[4])*942/2)
        cor = [xmin,ymax,xmax,ymin]
        corlis.append(cor)

    with open(f'/content/gdrive/MyDrive/AI競賽_運算/data/STAS_YOLOv5_PS/labels/val/{num:08}.txt','r') as RL:
      corlisL = []
      for line in RL.readlines():
        wordL = line.split(' ')
        xminL = round(float(wordL[1])*1716-float(wordL[3])*1716/2)
        yminL = round(float(wordL[2])*942-float(wordL[4])*942/2)
        xmaxL = round(float(wordL[1])*1716+float(wordL[3])*1716/2)
        ymaxL = round(float(wordL[2])*942+float(wordL[4])*942/2)
        corL = [xminL,ymaxL,xmaxL,yminL]
        corlisL.append(corL)

    img = cv2.imread(f'/content/gdrive/MyDrive/AI競賽_運算/data/STAS_YOLOv5_PS/images/val/{num:08}.jpg')
    for cor in corlis:
      cv2.rectangle(img, (cor[0],cor[1]), (cor[2],cor[3]),(0,0,255),2)
    for corL in corlisL:
      cv2.rectangle(img, (corL[0],corL[1]), (corL[2],corL[3]),(0,255,0),2)
    cv2.imwrite(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/{test_name}/{num:08}_labeled.jpg',img)

In [ ]:
P_G('test_for_label')

In [ ]:
print(len(os.listdir(f'/content/gdrive/MyDrive/AI競賽_運算/data/val_labeled/test_for_label')))

105
